# Installations and Imports



In [251]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install pandas



In [252]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [253]:
# pip install selenium

In [254]:
pip install webdriver-manager

## Attempt to Scrape from Premier League Site


In [255]:
def get_match_data(url):
    # Send a request to the webpage
    page = requests.get(url)

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')

    # Extract relevant data
    matches = []
    for match_container in soup.find_all('li', class_='matchFixtureContainer'):
        team_home = match_container['data-home']
        team_away = match_container['data-away']

        score_home_element = match_container.find('span', class_='score-home')
        score_away_element = match_container.find('span', class_='score-away')

        score_home = score_home_element.text.strip() if score_home_element else 'N/A'
        score_away = score_away_element.text.strip() if score_away_element else 'N/A'
        score = f"{score_home} - {score_away}"
        matches.append([team_home, team_away, score])

    # Convert to a DataFrame
    df = pd.DataFrame(matches, columns=['Home Team', 'Away Team', 'Score'])
    return df

# Example URL structure
base_url = 'https://www.premierleague.com/results'
all_matches = []

 # Adjust the range based on the number of pages
url = base_url
matches_df = get_match_data(url)
all_matches.append(matches_df)
print(url)

# Combine all match data into a single DataFrame
all_matches_df = pd.concat(all_matches, ignore_index=True)
all_matches_df.head()

https://www.premierleague.com/results


,Home Team,Away Team,Score


In [256]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time


# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL of the Premier League results page
url = 'https://www.premierleague.com/results'

# Open the webpage
driver.get(url)

# Wait for the page to load completely
time.sleep(5)  # Adjust the sleep time if necessary

# Find all the date containers
date_containers = driver.find_elements(By.CLASS_NAME, 'fixtures__date-container')

# Iterate over each date container to extract the date and matches
for date_container in date_containers:
    date_content = date_container.find_element(By.CLASS_NAME, 'fixtures__matches-list')
    if date_content:
        date_text = date_content.text.strip()
        print(f"Date: {date_text}")

    # Find the matches for each date
    matches = date_container.find_elements(By.CSS_SELECTOR, '[data-competition-matches-list]')
    for match in matches:
        match_text = match.text.strip()
        print(f"Match: {match_text}")

# Close the WebDriver
driver.quit()

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# URL of the Premier League results page

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all the date containers
    date_containers = soup.find_all('div', class_='col12')
    # Iterate over each date container to extract the date and matches
    for date_container in date_containers:
        date = date_container.find('div', class_='fixtures__matches-list')
        if date:
            date_text = date.get_text(strip=True)
            print(f"Date: {date_text}")

        # Find the matches for each date
        matches = date_container.find_all('div', {'data-competition-matches-list': True})
        for match in matches:
            match_text = match.get_text(strip=True)
            print(f"Match: {match_text}")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

## Scraping Real Data

In [ ]:
!pip install soccerdata
!pip install pandas

Using a pre-built scraping library

In [ ]:
import soccerdata as sd
import pandas as pd

# Define the leagues and seasons
leagues = ["ENG-Premier League", "ESP-La Liga", "ITA-Serie A", "GER-Bundesliga", "FRA-Ligue 1"]
seasons = ["2019/2020", "2020/2021", "2021/2022", "2022/2023"]

# Initialize an empty DataFrame to hold all match data
all_understat_match_data = pd.DataFrame()

# Loop through each league and season to scrape the data
for league in leagues:
    for season in seasons:
        print(f"Fetching data for {league} - {season}...")
        # Create an instance of the Understat scraper
        understat = sd.Understat(leagues=league, seasons=season)

        # Fetch player match stats (which includes match data)
        match_data = understat.read_team_match_stats()

        # Append the data to the main DataFrame
        all_understat_match_data = pd.concat([all_understat_match_data, match_data], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(all_understat_match_data.head())


In [ ]:
all_understat_match_data.info()

In [ ]:
all_understat_match_data.describe()

In [ ]:

# Define the leagues and seasons
leagues = ["ENG-Premier League", "ESP-La Liga", "ITA-Serie A", "GER-Bundesliga", "FRA-Ligue 1"]
seasons = ["2019/2020", "2020/2021", "2021/2022", "2022/2023"]

# Initialize an empty DataFrame to hold all match data
all_fd_match_data = pd.DataFrame()

# Loop through each league and season to scrape the data
for league in leagues:
    for season in seasons:
        print(f"Fetching data for {league} - {season}...")
        # Create an instance of the MatchHistory scraper
        mh = sd.MatchHistory(leagues=league, seasons=season)

        # Fetch match data
        match_data = mh.read_games()

        # Append the data to the main DataFrame
        all_fd_match_data = pd.concat([all_fd_match_data, match_data], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(all_fd_match_data.head())



In [ ]:
all_fd_match_data.info()

In [ ]:
all_fd_match_data.describe()

In [ ]:
combined_match_data = pd.concat([all_understat_match_data, all_fd_match_data], axis=1)

# List of duplicate columns to be removed
duplicate_columns = ['date', 'home_team', 'away_team', 'FTHG', 'FTAG']

# Removing duplicate columns
combined_match_data_cleaned = combined_match_data.drop(columns=duplicate_columns)

In [ ]:
combined_match_data_cleaned.to_csv('cleaned_combined_match_data.csv', index=False)

In [ ]:
import soccerdata as sd
import pandas as pd

# Define the leagues and seasons
leagues = ["ENG-Premier League", "ESP-La Liga", "ITA-Serie A", "GER-Bundesliga", "FRA-Ligue 1"]
seasons = ["2019/2020", "2020/2021", "2021/2022", "2022/2023"]

# Initialize an empty list to store dataframes
all_data = []

# Loop through each league and season
for league in leagues:
    for season in seasons:
        print(f"Fetching match-level possession data for {league} - {season}...")
        try:
            # Initialize the FBref reader
            fbref = sd.FBref(leagues=league, seasons=season)

            # Fetch match-level possession data
            possession_data = fbref.read_team_match_stats(stat_type="possession")

            # Append the DataFrame to the list
            all_data.append(possession_data)
        except Exception as e:
            print(f"Error fetching data for {league} - {season}: {e}")

# Combine all dataframes into one
combined_data = pd.concat(all_data, ignore_index=True)

# Save the combined data to a CSV file
combined_data.to_csv('combined_match_possession_data.csv', index=False)

# Display the combined data
print(combined_data.head())

In [ ]:
import soccerdata as sd
import pandas as pd
import time
import random
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Define the leagues and seasons
leagues = ["ENG-Premier League", "ESP-La Liga", "ITA-Serie A", "GER-Bundesliga", "FRA-Ligue 1"]
seasons = ["2019/2020", "2020/2021", "2021/2022", "2022/2023"]

# Initialize an empty list to store dataframes
all_data = []
missing_data = []

# Custom headers to mimic a web browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Function to create a requests session with retry strategy
def create_session():
    session = requests.Session()
    retry = Retry(
        total=5,
        backoff_factor=0.1,
        status_forcelist=[403, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    session.headers.update(headers)
    return session

# Create a global session
session = create_session()

# Monkey-patch requests to use the global session
requests.Session = lambda: session

# Loop through each league and season
for league in leagues:
    for season in seasons:
        print(f"Fetching match-level possession data for {league} - {season}...")
        try:
            # Initialize the FBref reader
            fbref = sd.FBref(leagues=league, seasons=season)

            # Fetch match-level possession data
            possession_data = fbref.read_team_match_stats(stat_type="possession")

            if possession_data.empty:
                print(f"No data for {league} - {season}")
                missing_data.append((league, season))
            else:
                # Append the DataFrame to the list
                all_data.append(possession_data)

            # Introduce a random delay between requests
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error fetching data for {league} - {season}: {e}")
            missing_data.append((league, season))

# Combine all dataframes into one
combined_data = pd.concat(all_data, ignore_index=True)

# Save the combined data to a CSV file
combined_data.to_csv('combined_match_possession_data.csv', index=False)

# Save missing data information to a CSV file
missing_data_df = pd.DataFrame(missing_data, columns=['League', 'Season'])
missing_data_df.to_csv('missing_data_info.csv', index=False)

# Display the combined data and missing data information
print("Combined Data Sample:")
print(combined_data.head())

print("Missing Data Information:")
print(missing_data_df)


In [ ]:
combined_data

In [ ]:
combined_data["round"].value_counts()

Custon FBREF Table Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_fbref_match_data(url):
    # Send a request to the webpage
    response = requests.get(url)
    response.raise_for_status()  # Check that the request was successful

    # Parse the webpage content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table with match data
    table = soup.find('table', {'id': 'results2023-202491_overall'})  # Adjust 'id' to match the actual id in FBref

    # Extract headers from the first header row that contains the column titles
    headers = [th.getText() for th in table.find('thead').find_all('th')][1:]

    # Extract rows
    rows = table.find('tbody').find_all('tr')
    match_data = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:  # Skip empty rows
            match_data.append([cell.getText() for cell in cells])
    print(headers)
    print(match_data)
    # Create a DataFrame
    df = pd.DataFrame(match_data, columns=headers)

    return df

# Example usage
url = 'https://fbref.com/en/comps/9/matchlogs/2022-2023/schedule/Premier-League-Scores-and-Fixtures'  # Replace with the desired URL
df = scrape_fbref_match_data(url)

# Display the DataFrame
print(df)

# Save to CSV
df.to_csv('fbref_match_data.csv', index=False)

In [ ]:
url = 'https://fbref.com/en/comps/9/matchlogs/2022-2023/schedule/Premier-League-Scores-and-Fixtures'  # Replace with the desired URL

# Send a request to the webpage
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Save the HTML content to a file
with open('fbref_page.html', 'w', encoding='utf-8') as file:
    file.write(response.text)

print("HTML content saved to fbref_page.html")

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables and print their IDs and headers
tables = soup.find_all('table')
for idx, table in enumerate(tables):
    table_id = table.get('id', 'No ID')
    header_rows = table.find_all('tr')
    if len(header_rows) > 1:  # Ensure there is at least one header row
        headers = [th.getText() for th in header_rows[1].find_all('th')]
    else:
        headers = [th.getText() for th in header_rows[0].find_all('th')]

    print(f"Table {idx+1}: ID = {table_id}")
    print("Headers:", headers)
    print()

Scraper for Football Data


In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the league standings page
league_standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

# Fetch the HTML content of the page
response = requests.get(league_standings_url)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
else:
    print("Successfully fetched the league standings page.")

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the team standings
standings_table = soup.find('table', {'id': 'results2023-202491_overall'})

# Extract the links to each team's page
team_links = []
for row in standings_table.find_all('tr'):
    team_cell = row.find('td', {'data-stat': 'team'})
    if team_cell:
        team_link = team_cell.find('a')['href']
        team_links.append('https://fbref.com' + team_link)

# Display the extracted team links
for link in team_links:
    print(link)

In [ ]:
def fetch_team_matchlogs(team_url):
    # Fetch the HTML content of the team's page
    response = requests.get(team_url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the match logs table
    matchlogs_table = soup.find('table', {'id': 'matchlogs_for'})
    if not matchlogs_table:
        print("Match logs table not found.")
        return None

    # Extract the table headers
    headers = [th.text for th in matchlogs_table.find('thead').find_all('th')]

    # Extract the table rows
    rows = matchlogs_table.find('tbody').find_all('tr')

    # Extract the data
    data = []
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if cols:
            data.append([col.text.strip() for col in cols])
    print(headers)
    print(data)
    # Create a DataFrame
    df = pd.DataFrame(data, columns=headers)

    return df

# Example usage: Fetch match logs for one team
team_url = 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats'
team_matchlogs = fetch_team_matchlogs(team_url)

# Display the extracted match logs
if team_matchlogs is not None:
    print(team_matchlogs.head())

In [ ]:
team_links = []
for row in standings_table.find_all('tr'):
    team_cell = row.find('td', {'data-stat': 'team'})
    if team_cell:
        team_link = team_cell.find('a')['href']
        team_links.append('https://fbref.com' + team_link)

# Initialize an empty DataFrame to hold all match logs
all_matchlogs = pd.DataFrame()

# Loop through each team URL to fetch match logs
for url in team_links:
    print(f"Fetching match logs for team: {url}")
    matchlogs_df = fetch_team_matchlogs(url)
    if matchlogs_df is not None:
        all_matchlogs = pd.concat([all_matchlogs, matchlogs_df], ignore_index=True)

# Display the combined match logs
print(all_matchlogs.head())

# Save the combined match logs to a CSV file
all_matchlogs.to_csv('combined_matchlogs.csv', index=False)

In [ ]:
!pip install retry

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from retry import retry

# URL of the league standings page
league_standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

# Fetch the HTML content of the page
response = requests.get(league_standings_url)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
else:
    print("Successfully fetched the league standings page.")

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the team standings
standings_table = soup.find('table', {'id': 'results2023-202491_overall'})

# Extract the links to each team's page
team_links = []
for row in standings_table.find_all('tr'):
    team_cell = row.find('td', {'data-stat': 'team'})
    if team_cell:
        team_link = team_cell.find('a')['href']
        team_links.append('https://fbref.com' + team_link)

# Initialize an empty DataFrame to hold all match data
all_team_match_data = pd.DataFrame()

# Relevant tabs to scrape (excluding miscellaneous and goalkeeper tabs)
relevant_tabs = {
    'shooting': 'shooting',
    'passing': 'passing',
    'passing_types': 'passing_types',
    'gca': 'gca',
    'defense': 'defense',
    'possession': 'possession'
}

# Function to extract table data
def extract_table_data(soup, table_id):
    table = soup.find('table', {'id': table_id})

    # Extract headers from the second row of thead
    headers = [header.text for header in table.find('thead').find_all('tr')[-1].find_all('th')]

    # Extract rows
    rows = []
    for row in table.find('tbody').find_all('tr'):
        cells = row.find_all(['th', 'td'])
        row_data = [cell.text.strip() for cell in cells]
        if row_data:
            rows.append(row_data)

    # Ensure all rows have the same number of columns as headers
    max_cols = len(headers)
    rows = [row[:max_cols] for row in rows]

    return pd.DataFrame(rows, columns=headers)

@retry(tries=5, delay=10, backoff=2)
def fetch_url(url):
    response = requests.get(url)
    if response.status_code == 429:
        raise Exception("Rate limit exceeded")
    return response

# Loop through each team link to scrape the match data
for team_url in team_links:
    print(f"Fetching data from {team_url}...")
    response = fetch_url(team_url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        continue

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract team name and ID from the URL
    base_url = team_url.rsplit('/', 2)[0]
    team_id = team_url.split('/')[-2]
    team_name = team_url.split('/')[-1].replace('-Stats', '')

    # Construct the URL for the 2023-2024 season
    season = '2023-2024'
    season_url = f'{base_url}/{team_id}/{season}/matchlogs/all_comps/'

    # Extract data from the main "matchlogs_for" table
    team_match_data = extract_table_data(soup, 'matchlogs_for')

    # Loop through each relevant tab to extract data
    for tab, tab_name in relevant_tabs.items():
        tab_url = f'{season_url}{tab}/{team_name}-Match-Logs-All-Competitions'
        print(f"Fetching data from {tab_url}...")

        tab_response = fetch_url(tab_url)
        if tab_response.status_code != 200:
            print(f"Failed to retrieve the page for tab {tab}. Status code: {tab_response.status_code}")
            continue

        # Parse the HTML content of the tab
        tab_soup = BeautifulSoup(tab_response.content, 'html.parser')

        # Extract table data from the tab (for and against)
        tab_match_data_for = extract_table_data(tab_soup, 'matchlogs_for')
        tab_match_data_against = extract_table_data(tab_soup, 'matchlogs_against')

        # Ensure the columns needed for merging exist in the dataframes
        common_columns = ['Date', 'Opponent', 'Venue', 'Result']
        for col in common_columns:
            if col not in tab_match_data_for.columns:
                print(f"Warning: Column {col} not found in {tab_name}_for")
                tab_match_data_for[col] = None
            if col not in tab_match_data_against.columns:
                print(f"Warning: Column {col} not found in {tab_name}_against")
                tab_match_data_against[col] = None

        # Merge the data from the tab with the main match data
        team_match_data = team_match_data.merge(tab_match_data_for, how='left', on=common_columns, suffixes=('', f'_{tab_name}_for'))
        team_match_data = team_match_data.merge(tab_match_data_against, how='left', on=common_columns, suffixes=('', f'_{tab_name}_against'))

    # Append the data to the main DataFrame
    all_team_match_data = pd.concat([all_team_match_data, team_match_data], ignore_index=True)

    # **Add a delay to avoid hitting rate limits**
    time.sleep(5)

# Remove duplicate columns
all_team_match_data = all_team_match_data.loc[:, ~all_team_match_data.columns.duplicated()]

# Display the first few rows of the combined DataFrame
print(all_team_match_data.head())

# Save the combined DataFrame to a CSV file
all_team_match_data.to_csv('all_team_match_data_combined.csv', index=False)


Using an API to Fetch Data

In [ ]:
import requests

# Replace with your actual API key
API_KEY = 'b7f5165795msh09ec88a04ad7216p1ccac8jsna49f64cb2496'
BASE_URL = 'https://api-football-v1.p.rapidapi.com/v3'

# Headers for authentication
headers = {
    'X-RapidAPI-Key': API_KEY,
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}

# Example endpoint to get standings data for a league
endpoint = f'{BASE_URL}/standings'

# Parameters for the request
params = {
    'league': '39',  # Premier League ID
    'season': '2023'  # Season
}

# Make the API request
response = requests.get(endpoint, headers=headers, params=params)
data = response.json()

# Print the standings data
print(data)

In [ ]:
response.json()

In [ ]:
# Extract the standings data
standings_data = data['response'][0]['league']['standings'][0]

# Normalize the standings data
standings_flat = pd.json_normalize(standings_data)

# Further normalize nested fields: team, all, home, away
if 'team' in standings_flat.columns:
    team_data = pd.json_normalize(standings_flat['team'])
    standings_flat = standings_flat.drop(columns=['team']).join(team_data)

if 'all' in standings_flat.columns:
    all_data = pd.json_normalize(standings_flat['all'], sep='_')
    standings_flat = standings_flat.drop(columns=['all']).join(all_data)

if 'home' in standings_flat.columns:
    home_data = pd.json_normalize(standings_flat['home'], sep='_')
    standings_flat = standings_flat.drop(columns=['home']).join(home_data)

if 'away' in standings_flat.columns:
    away_data = pd.json_normalize(standings_flat['away'], sep='_')
    standings_flat = standings_flat.drop(columns=['away']).join(away_data)

# Display the flattened DataFrame
print(standings_flat.head())

In [ ]:
standings_flat

In [ ]:
#Fixtures
import requests
import time
import pandas as pd

# Replace with your actual API key
API_KEY = 'your_api_key'
BASE_URL = 'https://api-football-v1.p.rapidapi.com/v3'

# Headers for authentication
headers = {
    'X-RapidAPI-Key': API_KEY,
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}

# Example endpoint to get fixtures for the Premier League (League ID: 39) for the 2023-2024 season
endpoint = f'{BASE_URL}/fixtures'
params = {
    'league': '39',  # Premier League ID
    'season': '2023'  # Season
}

# Make the API request
response = requests.get(endpoint, headers=headers, params=params)
fixtures = response.json()['response']
print(f"Total fixtures: {len(fixtures)}")

# Function to fetch match data
def fetch_match_data(fixture_id):
    endpoint = f'{BASE_URL}/fixtures/statistics'
    params = {'fixture': fixture_id}
    response = requests.get(endpoint, headers=headers, params=params)
    return response.json()['response']

# Fetch detailed data in batches
batch_size = 10
all_match_data = []

for i in range(0, len(fixtures), batch_size):
    batch = fixtures[i:i + batch_size]
    for fixture in batch:
        fixture_id = fixture['fixture']['id']
        match_data = fetch_match_data(fixture_id)
        all_match_data.append(match_data)
        print(f"Fetched data for fixture ID: {fixture_id}")

    # Sleep to avoid rate limiting
    time.sleep(60)  # Adjust sleep time if necessary

# Print number of match data fetched
print(f"Total match data fetched: {len(all_match_data)}")

# Convert the match data to a DataFrame
match_data_df = pd.json_normalize(all_match_data)

# Save to a CSV file
match_data_df.to_csv('match_data_2023_2024.csv', index=False)

# Print the DataFrame
print(match_data_df.head())


# Obtaining Extensive Possession Data



In [257]:
import requests
import time
import pandas as pd

In [21]:
df_test = pd.read_csv('/content/combined_match_possession_data (1).csv')

In [22]:
df_test.head()

,date,round,day,venue,result,GF,GA,opponent,Poss,Touches,...,Carries.2,Carries.3,Carries.4,Carries.5,Carries.6,Carries.7,Receiving,Receiving.1,time,match_report
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Touches,...,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,NaN,NaN
1,2019-08-11,Matchweek 1,Sun,Away,W,1,0,Newcastle Utd,62.0,771,...,1136,20,20,4,15,13,561,47,14:00:00,/en/matches/1405a610/Newcastle-United-Arsenal-...
2,2019-08-17,Matchweek 2,Sat,Home,W,2,1,Burnley,67.0,693,...,1413,32,22,11,14,10,471,33,12:30:00,/en/matches/ff7eda21/Arsenal-Burnley-August-17...
3,2019-08-24,Matchweek 3,Sat,Away,L,1,3,Liverpool,48.0,693,...,1152,14,13,2,18,18,471,28,17:30:00,/en/matches/102b241e/Liverpool-Arsenal-August-...
4,2019-09-01,Matchweek 4,Sun,Home,D,2,2,Tottenham,55.0,627,...,1280,21,14,8,19,10,406,65,16:30:00,/en/matches/0b6b8aaf/North-London-Derby-Arsena...


In [23]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14904 entries, 0 to 14903
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          14903 non-null  object 
 1   round         14903 non-null  object 
 2   day           14903 non-null  object 
 3   venue         14903 non-null  object 
 4   result        14903 non-null  object 
 5   GF            14903 non-null  object 
 6   GA            14903 non-null  object 
 7   opponent      14903 non-null  object 
 8   Poss          14893 non-null  float64
 9   Touches       14658 non-null  object 
 10  Touches.1     14658 non-null  object 
 11  Touches.2     14658 non-null  object 
 12  Touches.3     14658 non-null  object 
 13  Touches.4     14658 non-null  object 
 14  Touches.5     14658 non-null  object 
 15  Touches.6     14658 non-null  object 
 16  Take-Ons      14658 non-null  object 
 17  Take-Ons.1    14658 non-null  object 
 18  Take-Ons.2    14658 non-nu

In [24]:
import re
# Define a function to extract league and date from the match report URL
# Define a function to extract league from the match report URL
def extract_league(url):
    if pd.isna(url):
        return None
    match = re.search(r'/matches/[^/]+/[^/]+-\w+-\d+-\d+-(?P<league>[^/]+)', url)
    if match:
        return match.group('league')
    return None

# Apply the function to the "match_report" column
df_test['extracted_league'] = df_test['match_report'].apply(lambda x: extract_league(x))

In [25]:
df_test["extracted_league"].value_counts()

extracted_league
Serie-A                     3042
Premier-League              3040
La-Liga                     3040
Ligue-1                     2838
Bundesliga                  2448
DFB-Pokal                    219
Champions-League             139
Europa-League                111
Europa-Conference-League      16
DFL-Supercup                   8
UEFA-Super-Cup                 2
Name: count, dtype: int64

In [26]:
# Filter for Big 5 European leagues
big_5_leagues = ["Premier-League", "La-Liga", "Serie-A", "Bundesliga", "Ligue-1"]
df_filtered = df_test[df_test['extracted_league'].isin(big_5_leagues)]

In [27]:
# Check for the extra game in Serie A
serie_a_games = df_filtered[df_filtered['extracted_league'] == 'Serie-A']
print(f"Number of Serie A games: {len(serie_a_games)}")
print(f"Unique games: {serie_a_games[['match_report']].nunique()}")

# Identify the duplicate game(s) in Serie A by comparing relevant columns
relevant_columns = ['date', 'match_report']  # Add other relevant columns if needed
duplicate_games = serie_a_games[serie_a_games.duplicated(subset=relevant_columns, keep=False)]
print("Duplicate games in Serie A:")
print(duplicate_games)

Number of Serie A games: 3042
Unique games: match_report    1521
dtype: int64
Duplicate games in Serie A:
            date         round  day venue result GF GA     opponent  Poss  \
6081  2019-08-25   Matchweek 1  Sun  Away      W  3  2         SPAL  59.0   
6082  2019-09-01   Matchweek 2  Sun  Home      L  2  3       Torino  58.0   
6083  2019-09-15   Matchweek 3  Sun  Away      W  2  1        Genoa  48.0   
6084  2019-09-22   Matchweek 4  Sun  Home      D  2  2   Fiorentina  64.0   
6085  2019-09-25   Matchweek 5  Wed  Away      W  2  0         Roma  53.0   
...          ...           ...  ...   ...    ... .. ..          ...   ...   
9118  2023-05-08  Matchweek 34  Mon  Home      W  2  0    Sampdoria  46.0   
9119  2023-05-14  Matchweek 35  Sun  Away      L  0  2   Fiorentina  55.0   
9120  2023-05-21  Matchweek 36  Sun  Home      L  0  1        Lazio  40.0   
9121  2023-05-27  Matchweek 37  Sat  Away      L  2  3  Salernitana  52.0   
9122  2023-06-04  Matchweek 38  Sun  Home      

In [265]:
duplicate_games

,date,round,day,venue,result,GF,GA,opponent,Poss,Touches,...,Carries.3,Carries.4,Carries.5,Carries.6,Carries.7,Receiving,Receiving.1,time,match_report,extracted_league
6081,2019-08-25,Matchweek 1,Sun,Away,W,3,2,SPAL,59.0,636,...,19,16,4,7,6,403,38,20:45:00,/en/matches/0326af7e/SPAL-Atalanta-August-25-2...,Serie-A
6082,2019-09-01,Matchweek 2,Sun,Home,L,2,3,Torino,58.0,574,...,27,20,8,22,8,349,45,20:45:00,/en/matches/e55b020e/Atalanta-Torino-September...,Serie-A
6083,2019-09-15,Matchweek 3,Sun,Away,W,2,1,Genoa,48.0,494,...,34,26,11,18,13,301,40,12:30:00,/en/matches/7fb132b4/Genoa-Atalanta-September-...,Serie-A
6084,2019-09-22,Matchweek 4,Sun,Home,D,2,2,Fiorentina,64.0,721,...,36,35,7,20,16,509,63,18:00:00,/en/matches/ab7e6f6a/Atalanta-Fiorentina-Septe...,Serie-A
6085,2019-09-25,Matchweek 5,Wed,Away,W,2,0,Roma,53.0,679,...,16,21,6,7,11,458,35,19:00:00,/en/matches/95b95a98/Roma-Atalanta-September-2...,Serie-A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9118,2023-05-08,Matchweek 34,Mon,Home,W,2,0,Sampdoria,46.0,585,...,12,12,2,11,7,393,31,18:30:00,/en/matches/1e4613ad/Udinese-Sampdoria-May-8-2...,Serie-A
9119,2023-05-14,Matchweek 35,Sun,Away,L,0,2,Fiorentina,55.0,557,...,15,11,5,11,7,385,22,15:00:00,/en/matches/81c773b1/Fiorentina-Udinese-May-14...,Serie-A
9120,2023-05-21,Matchweek 36,Sun,Home,L,0,1,Lazio,40.0,563,...,13,17,1,18,5,382,29,20:45:00,/en/matches/7caf50be/Udinese-Lazio-May-21-2023...,Serie-A
9121,2023-05-27,Matchweek 37,Sat,Away,L,2,3,Salernitana,52.0,568,...,7,4,0,13,7,369,33,15:00:00,/en/matches/cc5813d7/Salernitana-Udinese-May-2...,Serie-A


In [28]:
# Extract the season based on the date
def extract_season(date):
    date = pd.to_datetime(date)
    year = date.year
    month = date.month
    day = date.day
    if (month > 8) or (month == 8 and day > 3):  # After August 3
        season = year
    else:  # On or before August 3
        season = year - 1
    return f"{season}/{season + 1}"


serie_a_games['season'] = serie_a_games['date'].apply(extract_season)


# Count the number of games per opponent per season
games_per_opponent_season = serie_a_games.groupby(['season', 'opponent']).size().reset_index(name='game_count')

<ipython-input-28-d908478b9e0c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie_a_games['season'] = serie_a_games['date'].apply(extract_season)


In [267]:
serie_a_games

,date,round,day,venue,result,GF,GA,opponent,Poss,Touches,...,Carries.4,Carries.5,Carries.6,Carries.7,Receiving,Receiving.1,time,match_report,extracted_league,season
6081,2019-08-25,Matchweek 1,Sun,Away,W,3,2,SPAL,59.0,636,...,16,4,7,6,403,38,20:45:00,/en/matches/0326af7e/SPAL-Atalanta-August-25-2...,Serie-A,2019/2020
6082,2019-09-01,Matchweek 2,Sun,Home,L,2,3,Torino,58.0,574,...,20,8,22,8,349,45,20:45:00,/en/matches/e55b020e/Atalanta-Torino-September...,Serie-A,2019/2020
6083,2019-09-15,Matchweek 3,Sun,Away,W,2,1,Genoa,48.0,494,...,26,11,18,13,301,40,12:30:00,/en/matches/7fb132b4/Genoa-Atalanta-September-...,Serie-A,2019/2020
6084,2019-09-22,Matchweek 4,Sun,Home,D,2,2,Fiorentina,64.0,721,...,35,7,20,16,509,63,18:00:00,/en/matches/ab7e6f6a/Atalanta-Fiorentina-Septe...,Serie-A,2019/2020
6085,2019-09-25,Matchweek 5,Wed,Away,W,2,0,Roma,53.0,679,...,21,6,7,11,458,35,19:00:00,/en/matches/95b95a98/Roma-Atalanta-September-2...,Serie-A,2019/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9118,2023-05-08,Matchweek 34,Mon,Home,W,2,0,Sampdoria,46.0,585,...,12,2,11,7,393,31,18:30:00,/en/matches/1e4613ad/Udinese-Sampdoria-May-8-2...,Serie-A,2022/2023
9119,2023-05-14,Matchweek 35,Sun,Away,L,0,2,Fiorentina,55.0,557,...,11,5,11,7,385,22,15:00:00,/en/matches/81c773b1/Fiorentina-Udinese-May-14...,Serie-A,2022/2023
9120,2023-05-21,Matchweek 36,Sun,Home,L,0,1,Lazio,40.0,563,...,17,1,18,5,382,29,20:45:00,/en/matches/7caf50be/Udinese-Lazio-May-21-2023...,Serie-A,2022/2023
9121,2023-05-27,Matchweek 37,Sat,Away,L,2,3,Salernitana,52.0,568,...,4,0,13,7,369,33,15:00:00,/en/matches/cc5813d7/Salernitana-Udinese-May-2...,Serie-A,2022/2023


In [29]:
# Identify opponents with extra games
extra_games_opponents = games_per_opponent_season[games_per_opponent_season['game_count'] > 38]

print("Number of games per opponent per season in Serie A:")
print(games_per_opponent_season)

print("Opponents with extra games in Serie A:")
print(extra_games_opponents)



Number of games per opponent per season in Serie A:
       season    opponent  game_count
0   2019/2020    Atalanta          38
1   2019/2020     Bologna          38
2   2019/2020     Brescia          38
3   2019/2020    Cagliari          38
4   2019/2020  Fiorentina          38
..        ...         ...         ...
75  2022/2023   Sampdoria          38
76  2022/2023    Sassuolo          38
77  2022/2023      Spezia          39
78  2022/2023      Torino          38
79  2022/2023     Udinese          38

[80 rows x 3 columns]
Opponents with extra games in Serie A:
       season       opponent  game_count
65  2022/2023  Hellas Verona          39
77  2022/2023         Spezia          39


In [30]:
games_per_opponent_season[0:82]

,season,opponent,game_count
0,2019/2020,Atalanta,38
1,2019/2020,Bologna,38
2,2019/2020,Brescia,38
3,2019/2020,Cagliari,38
4,2019/2020,Fiorentina,38
...,...,...,...
75,2022/2023,Sampdoria,38
76,2022/2023,Sassuolo,38
77,2022/2023,Spezia,39
78,2022/2023,Torino,38


In [31]:
# Create new columns with 'home_' or 'away_' prefixes based on the venue
home_columns = df_filtered.columns[~df_filtered.columns.isin(['match_report', 'venue', 'extracted_league'])]
away_columns = home_columns

for col in home_columns:
    df_filtered[f'home_{col}'] = df_filtered.apply(lambda row: row[col] if row['venue'] == 'Home' else None, axis=1)
    df_filtered[f'away_{col}'] = df_filtered.apply(lambda row: row[col] if row['venue'] == 'Away' else None, axis=1)

# Group by 'match_report' and aggregate
aggregated_data = df_filtered.groupby('match_report').agg(
    {**{f'home_{col}': 'first' for col in home_columns}, **{f'away_{col}': 'first' for col in away_columns}}
).reset_index()

# Add 'extracted_league' back to the aggregated data
aggregated_data['extracted_league'] = df_filtered.groupby('match_report')['extracted_league'].first().values

<ipython-input-31-c53c1e8350fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'home_{col}'] = df_filtered.apply(lambda row: row[col] if row['venue'] == 'Home' else None, axis=1)
<ipython-input-31-c53c1e8350fa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'away_{col}'] = df_filtered.apply(lambda row: row[col] if row['venue'] == 'Away' else None, axis=1)
<ipython-input-31-c53c1e8350fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [32]:
aggregated_data

,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,home_opponent,home_Poss,home_Touches,...,away_Carries.2,away_Carries.3,away_Carries.4,away_Carries.5,away_Carries.6,away_Carries.7,away_Receiving,away_Receiving.1,away_time,extracted_league
0,/en/matches/00023357/Hoffenheim-Bayer-Leverkus...,2021-04-12,Matchweek 28,Mon,D,0,0,Leverkusen,46.0,676,...,949,18,10,6,23,9,552,40,20:30:00,Bundesliga
1,/en/matches/0006415c/Udinese-Atalanta-October-...,2022-10-09,Matchweek 9,Sun,D,2,2,Atalanta,59.0,675,...,571,12,16,5,16,10,292,43,15:00:00,Serie-A
2,/en/matches/00173ae0/Nantes-Lille-August-12-20...,2022-08-12,Matchweek 2,Fri,D,1,1,Lille,31.0,455,...,1580,27,23,6,10,14,546,56,21:00:00,Ligue-1
3,/en/matches/001d9fe1/Internazionale-Hellas-Ver...,2023-01-14,Matchweek 18,Sat,W,1,0,Hellas Verona,57.0,669,...,558,15,13,2,22,7,312,39,20:45:00,Serie-A
4,/en/matches/002c4e89/Tottenham-Hotspur-Newcast...,2020-09-27,Matchweek 3,Sun,D,1,1,Newcastle Utd,66.0,788,...,428,2,7,0,12,7,263,13,14:00:00,Premier-League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,/en/matches/ffb4946c/Aston-Villa-Manchester-Ci...,2020-01-12,Matchweek 22,Sun,L,1,6,Manchester City,30.0,500,...,1948,37,22,13,14,9,829,57,16:30:00,Premier-League
7200,/en/matches/ffc25afd/Getafe-Barcelona-October-...,2020-10-17,Matchweek 6,Sat,W,1,0,Barcelona,28.0,341,...,1532,27,16,1,16,16,554,50,21:00:00,La-Liga
7201,/en/matches/ffce86d4/Real-Sociedad-Alaves-Marc...,2022-03-13,Matchweek 28,Sun,W,1,0,Alavés,59.0,659,...,440,15,5,5,17,12,221,23,18:30:00,La-Liga
7202,/en/matches/ffde27ac/Real-Madrid-Girona-Octobe...,2022-10-30,Matchweek 12,Sun,D,1,1,Girona,58.0,761,...,835,15,8,1,15,10,425,26,16:15:00,La-Liga


In [33]:
aggregated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7204 entries, 0 to 7203
Data columns (total 64 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_report      7204 non-null   object 
 1   home_date         7204 non-null   object 
 2   home_round        7204 non-null   object 
 3   home_day          7204 non-null   object 
 4   home_result       7204 non-null   object 
 5   home_GF           7204 non-null   object 
 6   home_GA           7204 non-null   object 
 7   home_opponent     7204 non-null   object 
 8   home_Poss         7204 non-null   float64
 9   home_Touches      7202 non-null   object 
 10  home_Touches.1    7202 non-null   object 
 11  home_Touches.2    7202 non-null   object 
 12  home_Touches.3    7202 non-null   object 
 13  home_Touches.4    7202 non-null   object 
 14  home_Touches.5    7202 non-null   object 
 15  home_Touches.6    7202 non-null   object 
 16  home_Take-Ons     7202 non-null   object 


In [272]:
# Define the order for sorting by league
league_order = {
    "Premier-League": 1,
    "La-Liga": 4,
    "Bundesliga": 3,
    "Serie-A": 2,
    "Ligue-1": 5
}

# Map league names to their sorting order
aggregated_data['league_order'] = aggregated_data['extracted_league'].map(league_order)

# Convert 'home_date' and 'home_time' columns to datetime format
aggregated_data['home_date'] = pd.to_datetime(aggregated_data['home_date'])
aggregated_data['home_time'] = pd.to_datetime(aggregated_data['home_time'], format='%H:%M:%S').dt.time

# Sort the data by league, matchday, and time within each matchday
aggregated_data = aggregated_data.sort_values(by=['league_order', 'home_date', 'home_time'])

# Rename the columns
aggregated_data.rename(columns={'away_opponent': 'home_team', 'home_opponent': 'away_team'}, inplace=True)

# Remove the final game between Spezia and Hellas Verona that occurred on June 11, 2023
aggregated_data = aggregated_data[~((aggregated_data['home_team'] == 'Spezia') &
                                    (aggregated_data['away_team'] == 'Hellas Verona') &
                                    (aggregated_data['home_date'] == '2023-06-11'))]

# Drop the 'league_order' column as it is no longer needed
aggregated_data = aggregated_data.drop(columns=['league_order'])

aggregated_data['season_id'] = aggregated_data['home_date'].apply(extract_season)

In [273]:
aggregated_data


,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,away_team,home_Poss,home_Touches,...,away_Carries.3,away_Carries.4,away_Carries.5,away_Carries.6,away_Carries.7,away_Receiving,away_Receiving.1,away_time,extracted_league,season_id
4137,/en/matches/928467bd/Liverpool-Norwich-City-Au...,2019-08-09,Matchweek 1,Fri,W,4,1,Norwich City,57.0,627,...,12,7,4,12,15,304,26,20:00:00,Premier-League,2019/2020
3202,/en/matches/71c8a43e/West-Ham-United-Mancheste...,2019-08-10,Matchweek 1,Sat,L,0,5,Manchester City,43.0,523,...,37,26,7,9,13,490,54,12:30:00,Premier-League,2019/2020
1462,/en/matches/34b99058/Burnley-Southampton-Augus...,2019-08-10,Matchweek 1,Sat,W,3,0,Southampton,47.0,507,...,13,9,5,16,16,269,32,15:00:00,Premier-League,2019/2020
1577,/en/matches/38111659/Watford-Brighton-and-Hove...,2019-08-10,Matchweek 1,Sat,L,0,3,Brighton,49.0,562,...,15,10,2,19,6,364,30,15:00:00,Premier-League,2019/2020
4724,/en/matches/a802f51e/Crystal-Palace-Everton-Au...,2019-08-10,Matchweek 1,Sat,D,0,0,Everton,36.0,427,...,10,11,1,26,12,432,54,15:00:00,Premier-League,2019/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501,/en/matches/7cb0ddb9/Nice-Lyon-June-3-2023-Lig...,2023-06-03,Matchweek 38,Sat,W,3,1,Lyon,43.0,623,...,24,35,6,19,7,570,60,21:00:00,Ligue-1,2022/2023
4504,/en/matches/a00583c3/Troyes-Lille-June-3-2023-...,2023-06-03,Matchweek 38,Sat,D,1,1,Lille,34.0,433,...,38,36,7,10,6,536,60,21:00:00,Ligue-1,2022/2023
4622,/en/matches/a44cfa36/Auxerre-Lens-June-3-2023-...,2023-06-03,Matchweek 38,Sat,L,1,3,Lens,51.0,583,...,15,12,9,8,9,401,37,21:00:00,Ligue-1,2022/2023
4707,/en/matches/a744a9bd/Lorient-Strasbourg-June-3...,2023-06-03,Matchweek 38,Sat,W,2,1,Strasbourg,53.0,731,...,22,32,7,12,13,489,46,21:00:00,Ligue-1,2022/2023


In [34]:
normal_data = pd.read_csv("/content/cleaned_combined_match_data.csv")
len(normal_data)

7203

In [37]:
normal_data

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,1,2019,11643,87,79,NOR,LIV,0,0.4405,1,...,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99
1,1,2019,11645,73,238,SHE,BOU,1,1.5905,1,...,1.92,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92
2,1,2019,11646,92,74,SOU,BUR,0,1.5136,0,...,1.71,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02
3,1,2019,11647,78,72,EVE,CRY,1,1.6257,0,...,1.71,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93
4,1,2019,11649,82,71,AVL,TOT,0,0.2794,1,...,2.40,-1.50,2.10,1.70,2.18,1.77,2.21,1.87,2.08,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,5,2022,20021,168,167,ANG,NAN,0,0.4451,0,...,2.27,-1.00,1.84,2.06,1.96,1.94,1.96,2.14,1.85,2.00
7199,5,2022,20023,170,178,LYO,NIC,0,0.9199,1,...,3.20,0.00,1.85,2.05,1.87,2.04,2.06,2.08,1.92,1.94
7200,5,2022,20024,161,270,CLE,PSG,3,1.9162,3,...,3.65,-1.75,2.04,1.86,2.03,1.86,2.06,2.04,1.93,1.92
7201,5,2022,20026,177,166,MPL,REI,3,1.6378,3,...,3.07,-0.75,2.03,1.87,2.07,1.83,2.09,1.88,2.01,1.84


In [35]:
normal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Columns: 122 entries, league_id to AvgCAHA
dtypes: float64(92), int64(25), object(5)
memory usage: 6.7+ MB


In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # Count non-NaN entries per column
    non_nan_counts = normal_data.count()
    print("Non-NaN entries per column:")
    print(non_nan_counts)

    print("\nData types of each column:")
    data_types = normal_data.dtypes
    print(data_types)

Non-NaN entries per column:
league_id                7203
season_id                7203
game_id                  7203
home_team_id             7203
away_team_id             7203
away_team_code           7203
home_team_code           7203
away_points              7203
away_expected_points     7203
away_goals               7203
away_xg                  7203
away_np_xg               7203
away_np_xg_difference    7203
away_ppda                7203
away_deep_completions    7203
home_points              7203
home_expected_points     7203
home_goals               7203
home_xg                  7203
home_np_xg               7203
home_np_xg_difference    7203
home_ppda                7203
home_deep_completions    7203
FTR                      7203
HTHG                     7203
HTAG                     7203
HTR                      7203
referee                  1520
HS                       7203
AS                       7203
HST                      7203
AST                      7203
HF          

In [275]:
normal_data[normal_data["league_id"]==2]

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
3040,2,2019,13090,110,105,NAP,FIO,3,1.3776,4,...,2.15,0.75,1.75,2.05,1.81,2.11,1.94,2.14,1.80,2.09
3041,2,2019,13089,230,98,JUV,PAR,3,2.6293,1,...,2.15,1.50,1.77,2.02,1.81,2.11,1.93,2.12,1.83,2.04
3042,2,2019,13097,116,242,BSC,CAG,3,1.6983,1,...,1.83,-0.75,2.10,1.83,2.11,1.82,2.12,1.85,2.08,1.82
3043,2,2019,13093,95,101,GEN,ROM,1,0.4172,3,...,2.23,-0.75,1.97,1.96,1.98,1.94,1.99,2.20,1.92,1.97
3044,2,2019,13095,221,107,ATA,SPL,3,1.5071,3,...,2.11,0.25,2.11,1.82,2.10,1.83,2.13,1.84,2.08,1.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,2,2022,18952,107,271,MON,ATA,0,0.1576,2,...,1.88,0.00,1.86,2.07,1.88,2.04,2.04,2.11,1.89,1.97
4556,2,2022,18955,243,97,BOL,LEC,3,0.9613,3,...,1.88,-0.75,1.90,2.03,1.90,2.00,1.92,2.06,1.86,2.00
4557,2,2022,18957,105,102,SAM,NAP,0,0.4274,0,...,4.27,-2.75,1.91,2.02,1.85,2.03,1.98,2.06,1.86,1.99
4558,2,2022,18958,95,260,SPE,ROM,0,0.5283,1,...,1.81,-0.75,2.04,1.89,2.04,1.88,2.11,1.93,2.02,1.84


In [276]:
unique_teams = aggregated_data['home_team'].unique()

In [277]:
print(unique_teams)

['Liverpool' 'West Ham' 'Burnley' 'Watford' 'Crystal Palace' 'Bournemouth'
 'Tottenham' 'Newcastle Utd' 'Leicester City' 'Manchester Utd' 'Arsenal'
 'Norwich City' 'Aston Villa' 'Brighton' 'Southampton' 'Everton'
 'Manchester City' 'Sheffield Utd' 'Chelsea' 'Wolves' 'Fulham' 'West Brom'
 'Leeds United' 'Brentford' "Nott'ham Forest" 'Parma' 'Fiorentina'
 'Udinese' 'SPAL' 'Roma' 'Torino' 'Hellas Verona' 'Sampdoria' 'Cagliari'
 'Inter' 'Bologna' 'Milan' 'Juventus' 'Lazio' 'Sassuolo' 'Genoa' 'Lecce'
 'Atalanta' 'Napoli' 'Brescia' 'Spezia' 'Crotone' 'Benevento' 'Empoli'
 'Salernitana' 'Venezia' 'Monza' 'Cremonese' 'Bayern Munich' 'Wolfsburg'
 'Freiburg' 'Dortmund' 'Leverkusen' 'Werder Bremen' 'Gladbach'
 'Eint Frankfurt' 'Union Berlin' 'Köln' 'Paderborn 07' 'Düsseldorf'
 'Hoffenheim' 'Augsburg' 'Mainz 05' 'Schalke 04' 'RB Leipzig' 'Hertha BSC'
 'Stuttgart' 'Arminia' 'Bochum' 'Greuther Fürth' 'Athletic Club'
 'Celta Vigo' 'Valencia' 'Mallorca' 'Leganés' 'Villarreal' 'Alavés'
 'Espanyol' 'Bet

In [278]:
# List of league names
leagues = ["Premier-League", "La-Liga", "Bundesliga", "Serie-A", "Ligue-1"]

# Mapping for full league names
league_order = {
    "Premier-League": 1,
    "La-Liga": 4,
    "Bundesliga": 3,
    "Serie-A": 2,
    "Ligue-1": 5
}

# Iterate through the list of league names and print the codes and teams
for league in leagues:
    full_league_name = league_order[league]

    # Get team codes from normal_data
    team_codes = normal_data[normal_data['league_id'] == full_league_name]['home_team_code'].unique()

    # Get team names from aggregated_data
    team_names = aggregated_data[aggregated_data['extracted_league'] == league]['home_team'].unique()

    print(f"League: {league}")
    print(f"Team Codes: {team_codes}")
    print(f"Team Names: {team_names}")
    print()

# Function to resolve specific conflicts
def resolve_conflict(row):
    if row['league_id'] == 4:  # La Liga
        if row['home_team_id'] == 231:  # Valladolid
            row['home_team_code'] = 'VLL'
        elif row['home_team_id'] == 146:  # Valencia
            row['home_team_code'] = 'VAL'
        if row['away_team_id'] == 231:  # Valladolid
            row['away_team_code'] = 'VLL'
        elif row['away_team_id'] == 146:  # Valencia
            row['away_team_code'] = 'VAL'
    return row

# Apply the conflict resolution
normal_data = normal_data.apply(resolve_conflict, axis=1)


# Function to append the league number to the team code
def append_league_id(row):
    league_id = row['league_id']
    row['home_team_code'] = f"{row['home_team_code']}{league_id}"
    row['away_team_code'] = f"{row['away_team_code']}{league_id}"
    return row

# Applying the function to each row in the dataframe
normal_data = normal_data.apply(append_league_id, axis=1)

League: Premier-League
Team Codes: ['LIV' 'BOU' 'BUR' 'CRY' 'TOT' 'WAT' 'WHU' 'LEI' 'MUN' 'NEW' 'ARS' 'AVL'
 'BRI' 'EVE' 'MCI' 'NOR' 'SOU' 'CHE' 'SHE' 'WOL' 'FLH' 'WBA' 'LED' 'BRE'
 'NOT']
Team Names: ['Liverpool' 'West Ham' 'Burnley' 'Watford' 'Crystal Palace' 'Bournemouth'
 'Tottenham' 'Newcastle Utd' 'Leicester City' 'Manchester Utd' 'Arsenal'
 'Norwich City' 'Aston Villa' 'Brighton' 'Southampton' 'Everton'
 'Manchester City' 'Sheffield Utd' 'Chelsea' 'Wolves' 'Fulham' 'West Brom'
 'Leeds United' 'Brentford' "Nott'ham Forest"]

League: La-Liga
Team Codes: ['ATH' 'CEL' 'LEG' 'MAL' 'VAL' 'VIL' 'ALA' 'ATL' 'ESP' 'BET' 'GRA' 'LEV'
 'GET' 'OSA' 'RMA' 'BAR' 'SEV' 'SOC' 'EIB' 'CAD' 'HUE' 'ELC' 'RVL' 'ALM'
 'GIR']
Team Names: ['Athletic Club' 'Celta Vigo' 'Valencia' 'Mallorca' 'Leganés' 'Villarreal'
 'Alavés' 'Espanyol' 'Betis' 'Atlético Madrid' 'Granada' 'Levante'
 'Osasuna' 'Real Madrid' 'Getafe' 'Barcelona' 'Sevilla' 'Real Sociedad'
 'Eibar' 'Valladolid' 'Cádiz' 'Huesca' 'Elche' 'Rayo Va

In [90]:
team_mapping = {
    'LIV1': 'Liverpool',
    'BOU1': 'Bournemouth',
    'BUR1': 'Burnley',
    'CRY1': 'Crystal Palace',
    'TOT1': 'Tottenham',
    'WAT1': 'Watford',
    'WHU1': 'West Ham',
    'LEI1': 'Leicester City',
    'MUN1': 'Manchester Utd',
    'NEW1': 'Newcastle Utd',
    'ARS1': 'Arsenal',
    'AVL1': 'Aston Villa',
    'BRI1': 'Brighton',
    'EVE1': 'Everton',
    'MCI1': 'Manchester City',
    'NOR1': 'Norwich City',
    'SOU1': 'Southampton',
    'CHE1': 'Chelsea',
    'SHE1': 'Sheffield Utd',
    'WOL1': 'Wolves',
    'FLH1': 'Fulham',
    'WBA1': 'West Brom',
    'LED1': 'Leeds United',
    'BRE1': 'Brentford',
    'NOT1': "Nott'ham Forest",
    'ATH4': 'Athletic Club',
    'CEL4': 'Celta Vigo',
    'LEG4': 'Leganés',
    'MAL4': 'Mallorca',
    'VAL4': 'Valencia',
    'VIL4': 'Villarreal',
    'ALA4': 'Alavés',
    'ATL4': 'Atlético Madrid',
    'ESP4': 'Espanyol',
    'BET4': 'Betis',
    'GRA4': 'Granada',
    'LEV4': 'Levante',
    'GET4': 'Getafe',
    'OSA4': 'Osasuna',
    'RMA4': 'Real Madrid',
    'BAR4': 'Barcelona',
    'SEV4': 'Sevilla',
    'SOC4': 'Real Sociedad',
    'VLL4': 'Valladolid',
    'EIB4': 'Eibar',
    'CAD4': 'Cádiz',
    'HUE4': 'Huesca',
    'ELC4': 'Elche',
    'RVL4': 'Rayo Vallecano',
    'ALM4': 'Almería',
    'GIR4': 'Girona',
    'BAY3': 'Bayern Munich',
    'LEV3': 'Leverkusen',
    'DOR3': 'Dortmund',
    'BMG3': 'Gladbach',
    'FRE3': 'Freiburg',
    'WER3': 'Werder Bremen',
    'WOL3': 'Wolfsburg',
    'EIN3': 'Eint Frankfurt',
    'UNI3': 'Union Berlin',
    'COL3': 'Köln',
    'AUG3': 'Augsburg',
    'FOR3': 'Düsseldorf',
    'HOF3': 'Hoffenheim',
    'MAI3': 'Mainz 05',
    'PAD3': 'Paderborn 07',
    'SCH3': 'Schalke 04',
    'HER3': 'Hertha BSC',
    'RBL3': 'RB Leipzig',
    'STU3': 'Stuttgart',
    'ARM3': 'Arminia',
    'BOC3': 'Bochum',
    'GRE3': 'Greuther Fürth',
    'FIO2': 'Fiorentina',
    'PAR2': 'Parma',
    'CAG2': 'Cagliari',
    'ROM2': 'Roma',
    'SPL2': 'SPAL',
    'SAM2': 'Sampdoria',
    'TOR2': 'Torino',
    'UDI2': 'Udinese',
    'VER2': 'Hellas Verona',
    'INT2': 'Inter',
    'BOL2': 'Bologna',
    'MIL2': 'Milan',
    'JUV2': 'Juventus',
    'ATA2': 'Atalanta',
    'GEN2': 'Genoa',
    'LAZ2': 'Lazio',
    'LEC2': 'Lecce',
    'SAS2': 'Sassuolo',
    'NAP2': 'Napoli',
    'BSC2': 'Brescia',
    'CRO2': 'Crotone',
    'SPE2': 'Spezia',
    'BEN2': 'Benevento',
    'EMP2': 'Empoli',
    'SAL2': 'Salernitana',
    'VEN2': 'Venezia',
    'MON2': 'Monza',
    'CRE2': 'Cremonese',
    'MON5': 'Monaco',
    'ANG5': 'Angers',
    'BRE5': 'Brest',
    'DJN5': 'Dijon',
    'MAR5': 'Marseille',
    'MPL5': 'Montpellier',
    'NIC5': 'Nice',
    'LIL5': 'Lille',
    'PSG5': 'Paris S-G',
    'STR5': 'Strasbourg',
    'LYO5': 'Lyon',
    'AMI5': 'Amiens',
    'BOR5': 'Bordeaux',
    'MET5': 'Metz',
    'NAN5': 'Nantes',
    'NIM5': 'Nîmes',
    'TOU5': 'Toulouse',
    'REI5': 'Reims',
    'REN5': 'Rennes',
    'SEN5': 'Saint-Étienne',
    'LOR5': 'Lorient',
    'LEN5': 'Lens',
    'TRO5': 'Troyes',
    'CLE5': 'Clermont Foot',
    'AJA5': 'Ajaccio',
    'AUX5': 'Auxerre'
}


In [280]:
normal_data

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,1,2019,11643,87,79,NOR1,LIV1,0,0.4405,1,...,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99
1,1,2019,11645,73,238,SHE1,BOU1,1,1.5905,1,...,1.92,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92
2,1,2019,11646,92,74,SOU1,BUR1,0,1.5136,0,...,1.71,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02
3,1,2019,11647,78,72,EVE1,CRY1,1,1.6257,0,...,1.71,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93
4,1,2019,11649,82,71,AVL1,TOT1,0,0.2794,1,...,2.40,-1.50,2.10,1.70,2.18,1.77,2.21,1.87,2.08,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,5,2022,20021,168,167,ANG5,NAN5,0,0.4451,0,...,2.27,-1.00,1.84,2.06,1.96,1.94,1.96,2.14,1.85,2.00
7199,5,2022,20023,170,178,LYO5,NIC5,0,0.9199,1,...,3.20,0.00,1.85,2.05,1.87,2.04,2.06,2.08,1.92,1.94
7200,5,2022,20024,161,270,CLE5,PSG5,3,1.9162,3,...,3.65,-1.75,2.04,1.86,2.03,1.86,2.06,2.04,1.93,1.92
7201,5,2022,20026,177,166,MPL5,REI5,3,1.6378,3,...,3.07,-0.75,2.03,1.87,2.07,1.83,2.09,1.88,2.01,1.84


In [281]:
normal_data[normal_data["home_team_code"]=="EVE1"]

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
13,1,2019,11656,72,90,WAT1,EVE1,0,1.6630,0,...,2.07,-0.75,2.11,1.82,2.12,1.81,2.16,1.87,2.07,1.83
39,1,2019,11681,72,229,WOL1,EVE1,0,1.7717,2,...,1.63,-0.50,2.02,1.77,2.08,1.85,2.11,1.86,2.07,1.82
52,1,2019,11698,72,238,SHE1,EVE1,3,0.7022,2,...,1.80,-0.75,2.05,1.88,2.05,1.88,2.06,2.00,2.01,1.88
64,1,2019,11711,72,88,MCI1,EVE1,3,1.8486,3,...,2.43,1.50,1.72,2.07,1.81,2.14,1.96,2.17,1.83,2.05
84,1,2019,11727,72,81,WHU1,EVE1,0,0.2537,0,...,2.33,-0.50,1.72,2.07,1.79,2.15,2.01,2.17,1.80,2.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,1,2022,18487,72,82,TOT1,EVE1,1,1.9082,1,...,1.71,0.50,1.86,2.07,1.88,2.06,1.88,2.17,1.81,2.08
1438,1,2022,18504,72,228,FLH1,EVE1,3,1.1410,3,...,1.66,-0.25,1.93,1.97,1.93,1.99,1.98,2.08,1.89,1.97
1461,1,2022,18522,72,86,NEW1,EVE1,3,2.7298,4,...,1.82,0.50,2.06,1.87,2.10,1.84,2.10,1.90,2.05,1.83
1495,1,2022,18557,72,88,MCI1,EVE1,3,1.1956,3,...,2.28,1.50,1.73,2.08,1.81,2.12,1.88,2.19,1.81,2.07


In [282]:
normal_data[(normal_data["home_team_code"] == "VAL4") & (normal_data["away_team_code"] == "VAL4")]

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA


In [283]:
aggregated_data


,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,away_team,home_Poss,home_Touches,...,away_Carries.3,away_Carries.4,away_Carries.5,away_Carries.6,away_Carries.7,away_Receiving,away_Receiving.1,away_time,extracted_league,season_id
4137,/en/matches/928467bd/Liverpool-Norwich-City-Au...,2019-08-09,Matchweek 1,Fri,W,4,1,Norwich City,57.0,627,...,12,7,4,12,15,304,26,20:00:00,Premier-League,2019/2020
3202,/en/matches/71c8a43e/West-Ham-United-Mancheste...,2019-08-10,Matchweek 1,Sat,L,0,5,Manchester City,43.0,523,...,37,26,7,9,13,490,54,12:30:00,Premier-League,2019/2020
1462,/en/matches/34b99058/Burnley-Southampton-Augus...,2019-08-10,Matchweek 1,Sat,W,3,0,Southampton,47.0,507,...,13,9,5,16,16,269,32,15:00:00,Premier-League,2019/2020
1577,/en/matches/38111659/Watford-Brighton-and-Hove...,2019-08-10,Matchweek 1,Sat,L,0,3,Brighton,49.0,562,...,15,10,2,19,6,364,30,15:00:00,Premier-League,2019/2020
4724,/en/matches/a802f51e/Crystal-Palace-Everton-Au...,2019-08-10,Matchweek 1,Sat,D,0,0,Everton,36.0,427,...,10,11,1,26,12,432,54,15:00:00,Premier-League,2019/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3501,/en/matches/7cb0ddb9/Nice-Lyon-June-3-2023-Lig...,2023-06-03,Matchweek 38,Sat,W,3,1,Lyon,43.0,623,...,24,35,6,19,7,570,60,21:00:00,Ligue-1,2022/2023
4504,/en/matches/a00583c3/Troyes-Lille-June-3-2023-...,2023-06-03,Matchweek 38,Sat,D,1,1,Lille,34.0,433,...,38,36,7,10,6,536,60,21:00:00,Ligue-1,2022/2023
4622,/en/matches/a44cfa36/Auxerre-Lens-June-3-2023-...,2023-06-03,Matchweek 38,Sat,L,1,3,Lens,51.0,583,...,15,12,9,8,9,401,37,21:00:00,Ligue-1,2022/2023
4707,/en/matches/a744a9bd/Lorient-Strasbourg-June-3...,2023-06-03,Matchweek 38,Sat,W,2,1,Strasbourg,53.0,731,...,22,32,7,12,13,489,46,21:00:00,Ligue-1,2022/2023


In [284]:
aggregated_data["match_report"]

4137    /en/matches/928467bd/Liverpool-Norwich-City-Au...
3202    /en/matches/71c8a43e/West-Ham-United-Mancheste...
1462    /en/matches/34b99058/Burnley-Southampton-Augus...
1577    /en/matches/38111659/Watford-Brighton-and-Hove...
4724    /en/matches/a802f51e/Crystal-Palace-Everton-Au...
                              ...                        
3501    /en/matches/7cb0ddb9/Nice-Lyon-June-3-2023-Lig...
4504    /en/matches/a00583c3/Troyes-Lille-June-3-2023-...
4622    /en/matches/a44cfa36/Auxerre-Lens-June-3-2023-...
4707    /en/matches/a744a9bd/Lorient-Strasbourg-June-3...
6603    /en/matches/e9d9bb82/Brest-Rennes-June-3-2023-...
Name: match_report, Length: 7203, dtype: object

In [285]:
aggregated_data["season_id"]

4137    2019/2020
3202    2019/2020
1462    2019/2020
1577    2019/2020
4724    2019/2020
          ...    
3501    2022/2023
4504    2022/2023
4622    2022/2023
4707    2022/2023
6603    2022/2023
Name: season_id, Length: 7203, dtype: object

In [286]:
# Function to apply the mapping
def map_team_name_to_code(team_name):
    return team_mapping.get(team_name, team_name)  # If no mapping found, return the name itself
1
# Apply the mapping to aggregated_data
aggregated_data['home_team_code'] = aggregated_data['home_team'].apply(map_team_name_to_code)
aggregated_data['away_team_code'] = aggregated_data['away_team'].apply(map_team_name_to_code)

# Apply the mapping to normal_data if team names are in columns
if 'home_team_code' in normal_data.columns:
    normal_data['home_team_og_code'] = normal_data['home_team_code']
    normal_data['home_team_code'] = normal_data['home_team_code'].apply(map_team_name_to_code)
if 'away_team_code' in normal_data.columns:
    normal_data['away_team_og_code'] = normal_data['away_team_code']
    normal_data['away_team_code'] = normal_data['away_team_code'].apply(map_team_name_to_code)

# Convert the season format in normal_data to match aggregated_data
normal_data['season_id'] = normal_data['season_id'].astype(str) + '/' + (normal_data['season_id'] + 1).astype(str)

# Ensure the date columns are in the same format
aggregated_data['home_date'] = pd.to_datetime(aggregated_data['home_date'])
# normal_data does not have home_date, we can create it from the available data if needed

# Check for non-matching rows in aggregated_data
non_matching_agg = aggregated_data.merge(normal_data, on=['home_team_code', 'away_team_code', 'season_id'], how='left', indicator=True)
non_matching_agg = non_matching_agg[non_matching_agg['_merge'] == 'left_only']

# Check for non-matching rows in normal_data
non_matching_norm = normal_data.merge(aggregated_data, on=['home_team_code', 'away_team_code', 'season_id'], how='left', indicator=True)
non_matching_norm = non_matching_norm[non_matching_norm['_merge'] == 'left_only']

print(f"Non-matching rows in aggregated_data: {len(non_matching_agg)}")
print(f"Non-matching rows in normal_data: {len(non_matching_norm)}")

# Combine datasets based on home team, away team, and season
combined_df = pd.merge(
    aggregated_data,
    normal_data,
    left_on=['home_team_code', 'away_team_code', 'season_id'],
    right_on=['home_team_code', 'away_team_code', 'season_id'],
    suffixes=('_agg', '_norm')
)

# Drop the redundant columns from the combined dataframe
combined_df = combined_df.drop(columns=['home_team_code', 'away_team_code'])

# Print out the number of rows for inspection
print(f"Rows in aggregated_data: {len(aggregated_data)}")
print(f"Rows in normal_data: {len(normal_data)}")
print(f"Rows in combined_df: {len(combined_df)}")

# Save the combined data to a new CSV for further inspection if needed
combined_df.to_csv('combined_data.csv', index=False)

# Print out a sample of the combined data
print(combined_df.head())

<ipython-input-286-0205ef88248e>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  normal_data['home_team_og_code'] = normal_data['home_team_code']
<ipython-input-286-0205ef88248e>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  normal_data['away_team_og_code'] = normal_data['away_team_code']


Non-matching rows in aggregated_data: 0
Non-matching rows in normal_data: 0
Rows in aggregated_data: 7203
Rows in normal_data: 7203
Rows in combined_df: 7203
                                        match_report  home_date   home_round  \
0  /en/matches/928467bd/Liverpool-Norwich-City-Au... 2019-08-09  Matchweek 1   
1  /en/matches/71c8a43e/West-Ham-United-Mancheste... 2019-08-10  Matchweek 1   
2  /en/matches/34b99058/Burnley-Southampton-Augus... 2019-08-10  Matchweek 1   
3  /en/matches/38111659/Watford-Brighton-and-Hove... 2019-08-10  Matchweek 1   
4  /en/matches/a802f51e/Crystal-Palace-Everton-Au... 2019-08-10  Matchweek 1   

  home_day home_result home_GF home_GA        away_team  home_Poss  \
0      Fri           W       4       1     Norwich City       57.0   
1      Sat           L       0       5  Manchester City       43.0   
2      Sat           W       3       0      Southampton       47.0   
3      Sat           L       0       3         Brighton       49.0   
4      Sat 

In [287]:
combined_df

,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,away_team,home_Poss,home_Touches,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,home_team_og_code,away_team_og_code
0,/en/matches/928467bd/Liverpool-Norwich-City-Au...,2019-08-09,Matchweek 1,Fri,W,4,1,Norwich City,57.0,627,...,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99,LIV1,NOR1
1,/en/matches/71c8a43e/West-Ham-United-Mancheste...,2019-08-10,Matchweek 1,Sat,L,0,5,Manchester City,43.0,523,...,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92,WHU1,MCI1
2,/en/matches/34b99058/Burnley-Southampton-Augus...,2019-08-10,Matchweek 1,Sat,W,3,0,Southampton,47.0,507,...,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02,BUR1,SOU1
3,/en/matches/38111659/Watford-Brighton-and-Hove...,2019-08-10,Matchweek 1,Sat,L,0,3,Brighton,49.0,562,...,2.04,1.86,2.05,1.88,2.12,1.91,2.05,1.84,WAT1,BRI1
4,/en/matches/a802f51e/Crystal-Palace-Everton-Au...,2019-08-10,Matchweek 1,Sat,D,0,0,Everton,36.0,427,...,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93,CRY1,EVE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,/en/matches/7cb0ddb9/Nice-Lyon-June-3-2023-Lig...,2023-06-03,Matchweek 38,Sat,W,3,1,Lyon,43.0,623,...,1.85,2.05,1.87,2.04,2.06,2.08,1.92,1.94,NIC5,LYO5
7199,/en/matches/a00583c3/Troyes-Lille-June-3-2023-...,2023-06-03,Matchweek 38,Sat,D,1,1,Lille,34.0,433,...,1.95,1.95,1.93,1.88,2.01,2.00,1.95,1.90,TRO5,LIL5
7200,/en/matches/a44cfa36/Auxerre-Lens-June-3-2023-...,2023-06-03,Matchweek 38,Sat,L,1,3,Lens,51.0,583,...,1.90,2.00,1.93,1.97,1.97,2.02,1.90,1.94,AUX5,LEN5
7201,/en/matches/a744a9bd/Lorient-Strasbourg-June-3...,2023-06-03,Matchweek 38,Sat,W,2,1,Strasbourg,53.0,731,...,1.88,2.02,1.88,2.03,1.93,2.06,1.85,2.00,LOR5,STR5


In [288]:
combined_df["extracted_league"].value_counts()

extracted_league
Premier-League    1520
Serie-A           1520
La-Liga           1520
Ligue-1           1419
Bundesliga        1224
Name: count, dtype: int64

In [289]:
non_matching_norm

,league_id,season_id,game_id,home_team_id,away_team_id,away_team_code,home_team_code,away_points,away_expected_points,away_goals,...,away_Carries.3,away_Carries.4,away_Carries.5,away_Carries.6,away_Carries.7,away_Receiving,away_Receiving.1,away_time,extracted_league,_merge


In [290]:
non_matching_agg

,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,away_team,home_Poss,home_Touches,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,home_team_og_code,away_team_og_code,_merge


In [292]:
combined_df.to_csv("final_football_data.csv", index=False)

# Data Cleaning

In [93]:
import requests
import time
import pandas as pd
final_df = pd.read_csv("/content/final_football_data.csv", low_memory=False)

In [94]:
final_df

,match_report,home_date,home_round,home_day,home_result,home_GF,home_GA,away_team,home_Poss,home_Touches,home_Touches.1,home_Touches.2,home_Touches.3,home_Touches.4,home_Touches.5,home_Touches.6,home_Take-Ons,home_Take-Ons.1,home_Take-Ons.2,home_Take-Ons.3,home_Take-Ons.4,home_Carries,home_Carries.1,home_Carries.2,home_Carries.3,home_Carries.4,home_Carries.5,home_Carries.6,home_Carries.7,home_Receiving,home_Receiving.1,home_time,away_date,away_round,away_day,away_result,away_GF,away_GA,home_team,away_Poss,away_Touches,away_Touches.1,away_Touches.2,away_Touches.3,away_Touches.4,away_Touches.5,away_Touches.6,away_Take-Ons,away_Take-Ons.1,away_Take-Ons.2,away_Take-Ons.3,away_Take-Ons.4,away_Carries,away_Carries.1,away_Carries.2,away_Carries.3,away_Carries.4,away_Carries.5,away_Carries.6,away_Carries.7,away_Receiving,away_Receiving.1,away_time,extracted_league,season_id,league_id,game_id,home_team_id,away_team_id,away_points,away_expected_points,away_goals,away_xg,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_goals,home_xg,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,FTR,HTHG,HTAG,HTR,referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,home_team_og_code,away_team_og_code
0,/en/matches/928467bd/Liverpool-Norwich-City-Au...,2019-08-09,Matchweek 1,Fri,W,4,1,Norwich City,57.0,627.0,44.0,117.0,307.0,209.0,35.0,627.0,15.0,11.0,73.3,4.0,26.7,383.0,2189.0,1197.0,32.0,19.0,10.0,14.0,8.0,408.0,54.0,20:00:00,2019-08-09,Matchweek 1,Fri,L,1,4,Liverpool,43.0,534.0,125.0,254.0,203.0,79.0,14.0,534.0,16.0,10.0,62.5,5.0,31.3,302.0,1415.0,587.0,12.0,7.0,4.0,12.0,15.0,304.0,26.0,20:00:00,Premier-League,2019/2020,1,11643,87,79,0,0.4405,1,0.842407,0.842407,-1.392153,21.454545,5,3,2.3863,4,2.234560,2.234560,1.392153,5.942857,11,H,4,0,H,M Oliver,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.15,9.59,18.05,1.12,8.50,21.00,1.14,9.5,23.00,1.16,10.00,23.00,1.14,8.75,19.83,1.40,3.00,1.40,3.11,1.45,3.11,1.41,2.92,-2.25,1.96,1.94,1.97,1.95,1.97,2.00,1.94,1.94,1.14,9.50,21.00,1.14,9.00,20.00,1.15,8.00,18.00,1.14,10.43,19.63,1.11,9.50,21.00,1.14,9.50,23.00,1.16,10.50,23.00,1.14,9.52,19.18,1.30,3.50,1.34,3.44,1.36,3.76,1.32,3.43,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99,LIV1,NOR1
1,/en/matches/71c8a43e/West-Ham-United-Mancheste...,2019-08-10,Matchweek 1,Sat,L,0,5,Manchester City,43.0,523.0,52.0,149.0,251.0,123.0,7.0,523.0,12.0,8.0,66.7,4.0,33.3,309.0,1498.0,618.0,10.0,8.0,2.0,15.0,14.0,347.0,27.0,12:30:00,2019-08-10,Matchweek 1,Sat,W,5,0,West Ham,57.0,682.0,60.0,210.0,315.0,168.0,26.0,681.0,18.0,13.0,72.2,5.0,27.8,485.0,2861.0,1618.0,37.0,26.0,7.0,9.0,13.0,490.0,54.0,12:30:00,Premier-League,2019/2020,1,11644,81,88,3,2.6312,5,3.183770,2.422640,1.222340,6.935484,9,0,0.2522,0,1.200300,1.200300,-1.222340,16.400000,1,A,0,1,A,M Dean,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,11.68,6.53,1.26,13.00,6.00,1.24,12.00,6.5,1.25,13.00,6.75,1.29,11.84,6.28,1.25,1.44,2.75,1.49,2.77,1.51,2.77,1.48,2.65,1.75,2.00,1.90,2.02,1.90,2.02,1.92,1.99,1.89,12.00,7.00,1.25,11.00,6.00,1.26,11.00,6.10,1.25,11.11,6.68,1.27,11.00,6.50,1.24,12.00,6.50,1.25,13.00,7.00,1.29,11.14,6.46,1.26,1.40,3.00,1.43,3.03,1.50,3.22,1.41,2.91,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92,WHU1,MCI1
2,/en/matches/34b99058/Burnley-Southampton-Augus...,2019-08-10,Matchweek 1,Sat,W,3,0,Southampton,47.0,507.0,55.0,154.0,269.0,89.0,14.0,507.0,15.0,14.0,93.3,1.0,6.7,214.0,977.0,477.0,8.0,3.0,2.0,20.

In [95]:
# Count non-NaN entries per column
non_nan_counts = final_df.count()

# Check data types of each column
data_types = final_df.dtypes

# Temporarily set display options
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # Count non-NaN entries per column
    non_nan_counts = final_df.count()
    print("Non-NaN entries per column:")
    print(non_nan_counts)

    print("\nData types of each column:")
    data_types = final_df.dtypes
    print(data_types)

Non-NaN entries per column:
match_report             7203
home_date                7203
home_round               7203
home_day                 7203
home_result              7203
home_GF                  7203
home_GA                  7203
away_team                7203
home_Poss                7203
home_Touches             7201
home_Touches.1           7201
home_Touches.2           7201
home_Touches.3           7201
home_Touches.4           7201
home_Touches.5           7201
home_Touches.6           7201
home_Take-Ons            7201
home_Take-Ons.1          7201
home_Take-Ons.2          7201
home_Take-Ons.3          7201
home_Take-Ons.4          7201
home_Carries             7201
home_Carries.1           7201
home_Carries.2           7201
home_Carries.3           7201
home_Carries.4           7201
home_Carries.5           7201
home_Carries.6           7201
home_Carries.7           7201
home_Receiving           7201
home_Receiving.1         7201
home_time                7203
away_date   

In [96]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Columns: 186 entries, match_report to away_team_og_code
dtypes: float64(138), int64(28), object(20)
memory usage: 10.2+ MB


In [97]:
# Fill NaN values in numeric columns with the mean
numeric_cols = final_df.select_dtypes(include=['number']).columns
final_df[numeric_cols] = final_df[numeric_cols].fillna(final_df[numeric_cols].mean())

In [98]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

# Print all column names
print(list(final_df.columns))



['match_report', 'home_date', 'home_round', 'home_day', 'home_result', 'home_GF', 'home_GA', 'away_team', 'home_Poss', 'home_Touches', 'home_Touches.1', 'home_Touches.2', 'home_Touches.3', 'home_Touches.4', 'home_Touches.5', 'home_Touches.6', 'home_Take-Ons', 'home_Take-Ons.1', 'home_Take-Ons.2', 'home_Take-Ons.3', 'home_Take-Ons.4', 'home_Carries', 'home_Carries.1', 'home_Carries.2', 'home_Carries.3', 'home_Carries.4', 'home_Carries.5', 'home_Carries.6', 'home_Carries.7', 'home_Receiving', 'home_Receiving.1', 'home_time', 'away_date', 'away_round', 'away_day', 'away_result', 'away_GF', 'away_GA', 'home_team', 'away_Poss', 'away_Touches', 'away_Touches.1', 'away_Touches.2', 'away_Touches.3', 'away_Touches.4', 'away_Touches.5', 'away_Touches.6', 'away_Take-Ons', 'away_Take-Ons.1', 'away_Take-Ons.2', 'away_Take-Ons.3', 'away_Take-Ons.4', 'away_Carries', 'away_Carries.1', 'away_Carries.2', 'away_Carries.3', 'away_Carries.4', 'away_Carries.5', 'away_Carries.6', 'away_Carries.7', 'away_Rece

In [99]:
# Load your DataFrame
final_df = pd.read_csv("/content/final_football_data.csv", low_memory=False)

# Create a dictionary to map old column names to new column names
rename_dict = {
    'HTHG': 'Half_Time_Home_Goals',
    'HTAG': 'Half_Time_Away_Goals',
    'HTR': 'Half_Time_Result',
    'HS': 'Home_Shots',
    'AS': 'Away_Shots',
    'HST': 'Home_Shots_on_Target',
    'AST': 'Away_Shots_on_Target',
    'HF': 'Home_Fouls',
    'AF': 'Away_Fouls',
    'HC': 'Home_Corners',
    'AC': 'Away_Corners',
    'HY': 'Home_Yellow_Cards',
    'AY': 'Away_Yellow_Cards',
    'HR': 'Home_Red_Cards',
    'AR': 'Away_Red_Cards'
}

# Rename the columns
final_df.rename(columns=rename_dict, inplace=True)

In [100]:
rename_dict = {
    'B365H': 'Bet365_Home_Win',
    'B365D': 'Bet365_Draw',
    'B365A': 'Bet365_Away_Win',
    'BWH': 'Bwin_Home_Win',
    'BWD': 'Bwin_Draw',
    'BWA': 'Bwin_Away_Win',
    'IWH': 'Interwetten_Home_Win',
    'IWD': 'Interwetten_Draw',
    'IWA': 'Interwetten_Away_Win',
    'PSH': 'Pinnacle_Home_Win',
    'PSD': 'Pinnacle_Draw',
    'PSA': 'Pinnacle_Away_Win',
    'WHH': 'WilliamHill_Home_Win',
    'WHD': 'WilliamHill_Draw',
    'WHA': 'WilliamHill_Away_Win',
    'VCH': 'BetVictor_Home_Win',
    'VCD': 'BetVictor_Draw',
    'VCA': 'BetVictor_Away_Win',
    'MaxH': 'Max_Home_Win',
    'MaxD': 'Max_Draw',
    'MaxA': 'Max_Away_Win',
    'AvgH': 'Avg_Home_Win',
    'AvgD': 'Avg_Draw',
    'AvgA': 'Avg_Away_Win',
    'B365>2.5': 'Bet365_Over_2.5',
    'B365<2.5': 'Bet365_Under_2.5',
    'P>2.5': 'Pinnacle_Over_2.5',
    'P<2.5': 'Pinnacle_Under_2.5',
    'Max>2.5': 'Max_Over_2.5',
    'Max<2.5': 'Max_Under_2.5',
    'Avg>2.5': 'Avg_Over_2.5',
    'Avg<2.5': 'Avg_Under_2.5',
    'AHh': 'Asian_Handicap_Home',
    'B365AHH': 'Bet365_Asian_Handicap_Home',
    'B365AHA': 'Bet365_Asian_Handicap_Away',
    'PAHH': 'Pinnacle_Asian_Handicap_Home',
    'PAHA': 'Pinnacle_Asian_Handicap_Away',
    'MaxAHH': 'Max_Asian_Handicap_Home',
    'MaxAHA': 'Max_Asian_Handicap_Away',
    'AvgAHH': 'Avg_Asian_Handicap_Home',
    'AvgAHA': 'Avg_Asian_Handicap_Away',
    'B365CH': 'Bet365_Closing_Home_Win',
    'B365CD': 'Bet365_Closing_Draw',
    'B365CA': 'Bet365_Closing_Away_Win',
    'BWCH': 'Bwin_Closing_Home_Win',
    'BWCD': 'Bwin_Closing_Draw',
    'BWCA': 'Bwin_Closing_Away_Win',
    'IWCH': 'Interwetten_Closing_Home_Win',
    'IWCD': 'Interwetten_Closing_Draw',
    'IWCA': 'Interwetten_Closing_Away_Win',
    'PSCH': 'Pinnacle_Closing_Home_Win',
    'PSCD': 'Pinnacle_Closing_Draw',
    'PSCA': 'Pinnacle_Closing_Away_Win',
    'WHCH': 'WilliamHill_Closing_Home_Win',
    'WHCD': 'WilliamHill_Closing_Draw',
    'WHCA': 'WilliamHill_Closing_Away_Win',
    'VCCH': 'BetVictor_Closing_Home_Win',
    'VCCD': 'BetVictor_Closing_Draw',
    'VCCA': 'BetVictor_Closing_Away_Win',
    'MaxCH': 'Max_Closing_Home_Win',
    'MaxCD': 'Max_Closing_Draw',
    'MaxCA': 'Max_Closing_Away_Win',
    'AvgCH': 'Avg_Closing_Home_Win',
    'AvgCD': 'Avg_Closing_Draw',
    'AvgCA': 'Avg_Closing_Away_Win',
    'B365C>2.5': 'Bet365_Closing_Over_2.5',
    'B365C<2.5': 'Bet365_Closing_Under_2.5',
    'PC>2.5': 'Pinnacle_Closing_Over_2.5',
    'PC<2.5': 'Pinnacle_Closing_Under_2.5',
    'MaxC>2.5': 'Max_Closing_Over_2.5',
    'MaxC<2.5': 'Max_Closing_Under_2.5',
    'AvgC>2.5': 'Avg_Closing_Over_2.5',
    'AvgC<2.5': 'Avg_Closing_Under_2.5',
    'AHCh': 'Closing_Asian_Handicap_Home',
    'B365CAHH': 'Bet365_Closing_Asian_Handicap_Home',
    'B365CAHA': 'Bet365_Closing_Asian_Handicap_Away',
    'PCAHH': 'Pinnacle_Closing_Asian_Handicap_Home',
    'PCAHA': 'Pinnacle_Closing_Asian_Handicap_Away',
    'MaxCAHH': 'Max_Closing_Asian_Handicap_Home',
    'MaxCAHA': 'Max_Closing_Asian_Handicap_Away',
    'AvgCAHH': 'Avg_Closing_Asian_Handicap_Home',
    'AvgCAHA': 'Avg_Closing_Asian_Handicap_Away'
}
# Rename the columns
final_df.rename(columns=rename_dict, inplace=True)

In [101]:
# Create a list of columns to keep, excluding redundant columns and those with dots followed by numbers
columns_to_keep = [
    col for col in final_df.columns
    if not ('.' in col and col.split('.')[-1].isdigit())
    and col not in [
        'home_date', 'away_date', 'home_day', 'away_day', 'referee',
        'away_round', 'home_GF', 'home_GA', 'away_GF', 'away_GA', 'away_result', 'away_time',
        'league_id', 'game_id', 'home_team_id', 'away_team_id', 'ftr', 'match_report'
    ]
]

# Add the new 'date' and 'round' columns based on 'home_date' and 'home_round'
new_df = final_df[columns_to_keep].copy()
new_df['date'] = final_df['home_date']
new_df['round'] = final_df['home_round']

# Drop the 'home_round' column since it's now redundant
new_df.drop(columns=['home_round'], inplace=True)

# Fill NaN values in numeric columns with the mean
numeric_cols = new_df.select_dtypes(include=['number']).columns
new_df[numeric_cols] = new_df[numeric_cols].fillna(new_df[numeric_cols].mean())

# Fill NaN values in non-numeric columns with an empty string
non_numeric_cols = new_df.select_dtypes(exclude=['number']).columns
new_df[non_numeric_cols] = new_df[non_numeric_cols].fillna('')

# Display the updated DataFrame info to verify changes
new_df.info()

# Check the dataframe to ensure the operations are completed
print(new_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Columns: 118 entries, home_result to round
dtypes: float64(86), int64(20), object(12)
memory usage: 6.5+ MB
  home_result        away_team  home_Poss  home_Touches  home_Take-Ons  \
0           W     Norwich City       57.0         627.0           15.0   
1           L  Manchester City       43.0         523.0           12.0   
2           W      Southampton       47.0         507.0           15.0   
3           L         Brighton       49.0         562.0           15.0   
4           D          Everton       36.0         427.0           12.0   

   home_Carries  home_Receiving home_time       home_team  away_Poss  \
0         383.0           408.0  20:00:00       Liverpool       43.0   
1         309.0           347.0  12:30:00        West Ham       57.0   
2         214.0           218.0  15:00:00         Burnley       53.0   
3         325.0           344.0  15:00:00         Watford       51.0   
4         160

In [102]:
new_df

,home_result,away_team,home_Poss,home_Touches,home_Take-Ons,home_Carries,home_Receiving,home_time,home_team,away_Poss,away_Touches,away_Take-Ons,away_Carries,away_Receiving,extracted_league,season_id,away_points,away_expected_points,away_goals,away_xg,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_goals,home_xg,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,FTR,Half_Time_Home_Goals,Half_Time_Away_Goals,Half_Time_Result,Home_Shots,Away_Shots,Home_Shots_on_Target,Away_Shots_on_Target,Home_Fouls,Away_Fouls,Home_Corners,Away_Corners,Home_Yellow_Cards,Away_Yellow_Cards,Home_Red_Cards,Away_Red_Cards,Bet365_Home_Win,Bet365_Draw,Bet365_Away_Win,Bwin_Home_Win,Bwin_Draw,Bwin_Away_Win,Interwetten_Home_Win,Interwetten_Draw,Interwetten_Away_Win,Pinnacle_Home_Win,Pinnacle_Draw,Pinnacle_Away_Win,WilliamHill_Home_Win,WilliamHill_Draw,WilliamHill_Away_Win,BetVictor_Home_Win,BetVictor_Draw,BetVictor_Away_Win,Max_Home_Win,Max_Draw,Max_Away_Win,Avg_Home_Win,Avg_Draw,Avg_Away_Win,Asian_Handicap_Home,Bet365_Asian_Handicap_Home,Bet365_Asian_Handicap_Away,Pinnacle_Asian_Handicap_Home,Pinnacle_Asian_Handicap_Away,Max_Asian_Handicap_Home,Max_Asian_Handicap_Away,Avg_Asian_Handicap_Home,Avg_Asian_Handicap_Away,Bet365_Closing_Home_Win,Bet365_Closing_Draw,Bet365_Closing_Away_Win,Bwin_Closing_Home_Win,Bwin_Closing_Draw,Bwin_Closing_Away_Win,Interwetten_Closing_Home_Win,Interwetten_Closing_Draw,Interwetten_Closing_Away_Win,Pinnacle_Closing_Home_Win,Pinnacle_Closing_Draw,Pinnacle_Closing_Away_Win,WilliamHill_Closing_Home_Win,WilliamHill_Closing_Draw,WilliamHill_Closing_Away_Win,BetVictor_Closing_Home_Win,BetVictor_Closing_Draw,BetVictor_Closing_Away_Win,Max_Closing_Home_Win,Max_Closing_Draw,Max_Closing_Away_Win,Avg_Closing_Home_Win,Avg_Closing_Draw,Avg_Closing_Away_Win,Closing_Asian_Handicap_Home,Bet365_Closing_Asian_Handicap_Home,Bet365_Closing_Asian_Handicap_Away,Pinnacle_Closing_Asian_Handicap_Home,Pinnacle_Closing_Asian_Handicap_Away,Max_Closing_Asian_Handicap_Home,Max_Closing_Asian_Handicap_Away,Avg_Closing_Asian_Handicap_Home,Avg_Closing_Asian_Handicap_Away,home_team_og_code,away_team_og_code,date,round
0,W,Norwich City,57.0,627.0,15.0,383.0,408.0,20:00:00,Liverpool,43.0,534.0,16.0,302.0,304.0,Premier-League,2019/2020,0,0.4405,1,0.842407,0.842407,-1.392153,21.454545,5,3,2.3863,4,2.234560,2.234560,1.392153,5.942857,11,H,4,0,H,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.14,8.25,18.50,1.15,8.00,18.00,1.15,9.59,18.05,1.12,8.50,21.00,1.14,9.5,23.00,1.16,10.00,23.00,1.14,8.75,19.83,-2.25,1.96,1.94,1.97,1.95,1.97,2.00,1.94,1.94,1.14,9.50,21.00,1.14,9.00,20.00,1.15,8.00,18.00,1.14,10.43,19.63,1.11,9.50,21.00,1.14,9.50,23.00,1.16,10.50,23.00,1.14,9.52,19.18,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99,LIV1,NOR1,2019-08-09,Matchweek 1
1,L,Manchester City,43.0,523.0,12.0,309.0,347.0,12:30:00,West Ham,57.0,682.0,18.0,485.0,490.0,Premier-League,2019/2020,3,2.6312,5,3.183770,2.422640,1.222340,6.935484,9,0,0.2522,0,1.200300,1.200300,-1.222340,16.400000,1,A,0,1,A,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.50,5.75,1.26,11.00,6.10,1.25,11.68,6.53,1.26,13.00,6.00,1.24,12.00,6.5,1.25,13.00,6.75,1.29,11.84,6.28,1.25,1.75,2.00,1.90,2.02,1.90,2.02,1.92,1.99,1.89,12.00,7.00,1.25,11.00,6.00,1.26,11.00,6.10,1.25,11.11,6.68,1.27,11.00,6.50,1.24,12.00,6.50,1.25,13.00,7.00,1.29,11.14,6.46,1.26,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92,WHU1,MCI1,2019-08-10,Matchweek 1
2,W,Southampton,47.0,507.0,15.0,214.0,218.0,15:00:00,Burnley,53.0,556.0,11.0,230.0,269.0,Premier-League,2019/2020,0,1.5136,0,1.087520,1.087520,0.178279,5.500000,9,3,1.1422,3,0.909241,0.909241,-0.178279,9.277778,0,H,0,0,D,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.65,3.20,2.75,2.65,3.20,2.75,2.71,3.31,2.81,2.70,3.20,2.75,2.70,3.3,2.80,2.80,3.33,2.85,2.68,3.22,2.78,0.00,1.92,1.98,1.93,2.00,1.94,2.00,1.91,1.98,2.70,3.25,2.90,2.65,3.10,2.85,2.60,3.20,2.85,2.71,3.19,2.90,2.62,3.20,2.80,2.70,3.25,2.90,2.72,3.26,2.95,2.65,3.18,2.88,0.00,1.87,2.

In [103]:
# Create a DataFrame from the team_mapping
team_mapping_df = pd.DataFrame(list(team_mapping.items()), columns=['Mapped_Code', 'Team_Name'])

# Merge new_df with team_mapping_df to compare home_team and home_team_og_code
comparison_df = new_df[['home_team', 'home_team_og_code']].merge(
    team_mapping_df,
    left_on='home_team_og_code',
    right_on='Mapped_Code',
    how='left'
)

# Check if the home_team matches the mapped team name
comparison_df['Match'] = comparison_df['home_team'] == comparison_df['Team_Name']

# Display the comparison DataFrame
print(comparison_df.head(20))  # Display the first 20 rows for comparison

          home_team home_team_og_code Mapped_Code        Team_Name  Match
0         Liverpool              LIV1        LIV1        Liverpool   True
1          West Ham              WHU1        WHU1         West Ham   True
2           Burnley              BUR1        BUR1          Burnley   True
3           Watford              WAT1        WAT1          Watford   True
4    Crystal Palace              CRY1        CRY1   Crystal Palace   True
5       Bournemouth              BOU1        BOU1      Bournemouth   True
6         Tottenham              TOT1        TOT1        Tottenham   True
7     Newcastle Utd              NEW1        NEW1    Newcastle Utd   True
8    Leicester City              LEI1        LEI1   Leicester City   True
9    Manchester Utd              MUN1        MUN1   Manchester Utd   True
10          Arsenal              ARS1        ARS1          Arsenal   True
11     Norwich City              NOR1        NOR1     Norwich City   True
12      Aston Villa              AVL1 

In [104]:
comparison_df["Match"].value_counts()

Match
True    7203
Name: count, dtype: int64

In [105]:
new_df.to_csv("final_processed_football_data.csv", index=False)